In [1]:
import os
import shutil
import time
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from sklearn.model_selection  import train_test_split
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchvision import transforms
import torchvision.datasets as dset
from PIL import Image
from efficientnet_pytorch import EfficientNet
from torchvision.models import Inception3, inception_v3
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
import itertools
from sklearn.model_selection import KFold, StratifiedKFold

In [5]:
save_file = 'predict_kfold'

if not os.path.exists(save_file):
    os.mkdir(save_file)
    
save_model_file = os.path.join(save_file, 'save_model')
if not os.path.exists(save_model_file):
    os.mkdir(save_model_file)

In [4]:
train_data_path = 'D:/_kaggle_Plant Pathology 2020 dataset/Train_4_class'
train_df = pd.read_csv('D:/_kaggle_Plant Pathology 2020 dataset/train.csv')
train_label = train_df.iloc[:, 1:].values
train_label = train_label[:, 2] + train_label[:, 3] * 2 + train_label[:, 1] * 3

test_data_path = 'D:/_kaggle_Plant Pathology 2020 dataset/Test'

In [3]:
class kaggle_Dataset(Dataset):
    def __init__(self, data, transforms, mode):
        self.transforms = transforms
        self.mode = mode
        
        if mode == 'train' or mode == 'valid':
            data_path = []
            data_label = []
            
            for i in range(len(data)):
                class_eye = np.eye(4)
                if data['healthy'][i] == 1:
                    class_id = class_eye[0]
                    class_ = 'healthy'
                    
                elif data['multiple_diseases'][i] == 1:
                    class_id = class_eye[1]
                    class_ = 'multiple_diseases'

                elif data['rust'][i] == 1:
                    class_id = class_eye[2]
                    class_ = 'rust'

                elif data['scab'][i] == 1:
                    class_id = class_eye[3]
                    class_ = 'scab'

                data_label.append(class_id)
                
                image_path = os.path.join(train_data_path, class_, data['image_id'][i]+'.jpg')
                data_path.append(image_path) 
                
            self.data_path = data_path
            self.labels = np.array(data_label)            
        
        elif mode == 'test':
            x_test = []
            for image_id in os.listdir(data):
                test_path = os.path.join(data, image_id)
                x_test.append(test_path)
                
            self.data_path = x_test

    def __len__(self):
        return len(self.data_path)

    def __getitem__(self, idx):
        img = Image.open(self.data_path[idx]).convert("RGB")
        img = self.transforms(img)
        
        if self.mode == 'test':            
            return img
        
        else:
            labels = self.labels[idx]
            return img, labels

        
train_transform = transforms.Compose([transforms.Resize((256, 256)),
                                transforms.RandomHorizontalFlip(p=0.3),
                                transforms.RandomSizedCrop(224),
                                transforms.RandomRotation(10),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                     ])
        
valid_transform = transforms.Compose([transforms.Resize((256, 256)),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                   ])

test_transform = transforms.Compose([transforms.Resize((256, 256)),
                                transforms.RandomHorizontalFlip(p=0.3),
                                transforms.CenterCrop(224),
                                transforms.RandomRotation(10),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                     ])

def test_dataset_loader(test_data_path, TTA):
    if TTA == 1:
        test_dataset = kaggle_Dataset(test_data_path, transforms = valid_transform, mode = 'test')
        test_loader = DataLoader(test_dataset, batch_size, shuffle=False)
    else:
        test_dataset = kaggle_Dataset(test_data_path, transforms = test_transform, mode = 'test')
        test_loader = DataLoader(test_dataset, batch_size, shuffle=False)
    
    return test_loader


C:\ProgramData\Anaconda3\lib\site-packages\torchvision\transforms\transforms.py:562: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  warnings.warn("The use of the transforms.RandomSizedCrop transform is deprecated, " +


In [14]:
batch_size = 32
SEED = 42
N_FOLDS = 5
folds = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

In [2]:
model_name = 'efficientnet-b0'

In [8]:
def PlantModel():
    model = EfficientNet.from_pretrained(model_name)
    model._fc = nn.Linear(in_features=1280,out_features=4)
    model.cuda()
    
    return model

In [15]:
lr = 3e-4
num_epochs = 100
TTA = 8

In [31]:
def train_one_fold(i_fold, model, exp_lr_scheduler, optimizer, train_loader, valid_loader):
    train_acc = []
    train_loss = []
    val_acc = []
    val_loss = []

    train_fold_results = []

    for epoch in range(num_epochs):    
        ## training 
        start = time.time()
        model.train()
        exp_lr_scheduler.step()
        total_train = 0
        correct_train = 0
        total_train_loss = 0

        for batch_idx, (data, target) in enumerate(train_loader):
            data = Variable(data).cuda()
            target = Variable(target.float()).cuda()
            target_value = torch.max(target, 1)[1]

            optimizer.zero_grad()                   # Clear gradients
            output = model(data)                   # Forward propagation
            output_log = F.log_softmax(output, dim=1)
            train_loss_ = F.nll_loss(output_log, target_value, reduction='sum')

            train_loss_.backward()                   # Calculate gradients
            optimizer.step()                        # Update parameters

            predicted = torch.max(output_log.data, 1)[1]

            total_train += len(target_value)
            correct_train += sum((predicted == target_value).float())
            total_train_loss += train_loss_.item()

            if batch_idx % 10 == 0:
                print(f'Train Epoch: {epoch+1}/{num_epochs} [iter:{batch_idx+1}/{len(train_loader)}], acc:{correct_train / float((batch_idx + 1) * batch_size)}, loss:{total_train_loss/float((batch_idx + 1) * batch_size)}')

        train_acc_ = 100 * ((correct_train/ total_train).float())
        train_acc.append(train_acc_.item())
        train_loss.append(total_train_loss/ total_train)

        ## evaluate
        model.eval()
        total_val = 0
        corrent_val = 0
        total_val_loss = 0
        val_target_list = []
        val_predict_list = []

        with torch.no_grad():
            for data, target in valid_loader:
                data = Variable(data).cuda()
                target = Variable(target.float()).cuda()
                target_value = torch.max(target, 1)[1]
                val_target_list.append(target_value.cpu().numpy())

                val_output = model(data)
                val_output_log = F.log_softmax(val_output, dim=1)
                val_loss_ = F.nll_loss(val_output_log, target_value, reduction='sum')

                val_predicted = torch.max(val_output_log.data, 1)[1]
                val_predict_list.append(val_predicted.cpu().numpy())

                total_val += len(target_value)
                corrent_val += sum((val_predicted == target_value).float())
                total_val_loss += val_loss_.item()

        val_acc_ = 100 * ((corrent_val/ total_val).float())
        val_acc.append(val_acc_.item())
        val_loss.append(total_val_loss/ total_val)

        if(epoch==0):
            val_loss_min = val_loss_

        if val_loss_< val_loss_min:
            torch.save(model.state_dict(), os.path.join(save_model_file, 'model_best.pth'))
            val_loss_min = val_loss_
            val_target_value = val_target_list
            val_predict_value = val_predict_list

        print('Train Epoch: {}/{} Traing_Loss: {} Traing_acc: {:.6f}% \nVal_Loss: {} Val_accuracy: {:.6f}%'.format(epoch+1, num_epochs, train_loss[epoch], train_acc_, val_loss[epoch], val_acc_))
        end = time.time()
        print('1 epoch spends ', end-start, ' seconds')
    
    train_fold_results.append({
                            'fold' : i_fold, 
                            'train_acc': train_acc,
                            'train_loss': train_loss, 
                            'val_acc': val_acc,
                            'val_loss': val_loss,
                            'val_target_value': val_target_value,
                            'val_predict_value': val_predict_value
                                })
    
    return train_fold_results

In [39]:
def kfold_(TTA, test_data_path):
    train_results = []
    prediction_list = []
    total_predict = []
    test_loader = test_dataset_loader(test_data_path, TTA)

    for i_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df, train_label)):
        start = time.time()
        print("Fold {}/{}".format(i_fold + 1, N_FOLDS))

        valid = train_df.iloc[valid_idx]
        valid.reset_index(drop=True, inplace=True)

        train = train_df.iloc[train_idx]
        train.reset_index(drop=True, inplace=True)

        dataset_train = kaggle_Dataset(train, transforms=train_transform, mode='train')
        dataset_valid = kaggle_Dataset(valid, transforms=valid_transform, mode='valid')

        train_loader = DataLoader(dataset_train , batch_size, shuffle=True)
        valid_loader = DataLoader(dataset_valid , batch_size, shuffle=False)

        model = PlantModel()
        model.cuda()
        optimizer = optim.Adam(model.parameters(), lr)
        exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size = 30, gamma = 0.1)

        train_fold_results = train_one_fold(i_fold, model, exp_lr_scheduler, optimizer, train_loader, valid_loader)
        train_results += train_fold_results

        with torch.no_grad():
            model.load_state_dict(torch.load(os.path.join(save_model_file, 'model_best.pth')))
            model.eval()
            pred_exp_list = []

            print('***********************')
            print('start predict.')

            for tta in range(TTA):
                pred = []
                print(f'TTA {tta+1} start predict.')

                for inputs in test_loader:
                    pred_output = model(inputs.cuda())
                    pred_output_log = F.log_softmax(pred_output, dim=1)        
                    pred_exp = torch.exp(pred_output_log)

                    for pred_exp_idx in pred_exp:
                        pred.append(pred_exp_idx.cpu().numpy())

                pred_exp_list.append(pred)

            tta_ensem = []

            for model_idx in range(len(pred_exp_list)):
                if model_idx == 0:
                        tta_ensem += pred_exp_list[0]

                else:            
                    for pred_idx in range(len(pred_exp_list[0])):
                        tta_ensem[pred_idx] += pred_exp_list[model_idx][pred_idx]

            tta_ensem_ = [i/TTA for i in tta_ensem]

            total_predict.append(tta_ensem_)

        end = time.time()
        print(end-start, ' seconds')
        print('********************************')

    fold_ensem = []

    for fold_idx in range(len(total_predict)):
        if fold_idx == 0:
                fold_ensem += total_predict[0]

        else:            
            for pred_idx in range(len(total_predict[0])):
                fold_ensem[pred_idx] += total_predict[fold_idx][pred_idx]

    fold_ensem_ = [i/N_FOLDS for i in fold_ensem]

    pred_exp_list = pd.DataFrame(fold_ensem_, columns = ['healthy', 'multiple_diseases', 'rust', 'scab'])

    test_image_id = []
    for image_id in os.listdir(test_data_path):
        test_image_id.append(image_id.replace('.jpg',''))

    test_image_id = pd.DataFrame(test_image_id, columns = ['image_id'])

    predict_df = pd.concat([test_image_id, pred_exp_list], axis = 1)

    return predict_df, train_results

In [ ]:
predict_df, train_results = kfold_(TTA, test_data_path)

Fold 1/5
Train Epoch: 1/100 [iter:1/46], acc:0.28125, loss:1.3475992679595947
Train Epoch: 1/100 [iter:11/46], acc:0.6022727489471436, loss:1.0028332580219617
Train Epoch: 1/100 [iter:21/46], acc:0.6845238208770752, loss:0.8178515036900839
Train Epoch: 1/100 [iter:31/46], acc:0.7167338728904724, loss:0.7539981065257904
Train Epoch: 1/100 [iter:41/46], acc:0.7439023852348328, loss:0.6844092941865688
Train Epoch: 1/100 Traing_Loss: 0.6665515478244346 Traing_acc: 75.378265% 
Val_Loss: 0.3439388463542637 Val_accuracy: 89.918259%
1 epoch spends  10050.426097631454  seconds
Train Epoch: 2/100 [iter:1/46], acc:0.78125, loss:0.5451135635375977
Train Epoch: 2/100 [iter:11/46], acc:0.8352273106575012, loss:0.4400627220218832
Train Epoch: 2/100 [iter:21/46], acc:0.8392857313156128, loss:0.436382735768954
Train Epoch: 2/100 [iter:31/46], acc:0.8457661271095276, loss:0.44071348299903257
Train Epoch: 2/100 [iter:41/46], acc:0.8544207215309143, loss:0.4233915467814701
Train Epoch: 2/100 Traing_Loss: 

Train Epoch: 29/100 [iter:31/46], acc:0.9485886693000793, loss:0.14001519821824565
Train Epoch: 29/100 [iter:41/46], acc:0.9496951103210449, loss:0.14420264077986159
Train Epoch: 29/100 Traing_Loss: 0.14534839335957125 Traing_acc: 94.979370% 
Val_Loss: 0.234284150827808 Val_accuracy: 94.550407%
1 epoch spends  11468.273863077164  seconds
Train Epoch: 30/100 [iter:1/46], acc:0.875, loss:0.2100556343793869
Train Epoch: 30/100 [iter:11/46], acc:0.9602273106575012, loss:0.11968039179390128
Train Epoch: 30/100 [iter:21/46], acc:0.9598214626312256, loss:0.11908891619670958
Train Epoch: 30/100 [iter:31/46], acc:0.9536290168762207, loss:0.15121900314284908
Train Epoch: 30/100 [iter:41/46], acc:0.957317054271698, loss:0.13880198040023084
Train Epoch: 30/100 Traing_Loss: 0.14340678361619846 Traing_acc: 95.529572% 
Val_Loss: 0.2621039272329138 Val_accuracy: 92.643051%
1 epoch spends  11518.810731172562  seconds
Train Epoch: 31/100 [iter:1/46], acc:0.9375, loss:0.06442176550626755
Train Epoch: 31/

Val_Loss: 0.19176454797427725 Val_accuracy: 96.185287%
1 epoch spends  12173.746685028076  seconds
Train Epoch: 44/100 [iter:1/46], acc:1.0, loss:0.04275054484605789
Train Epoch: 44/100 [iter:11/46], acc:0.9715909361839294, loss:0.0684589415111325
Train Epoch: 44/100 [iter:21/46], acc:0.973214328289032, loss:0.07231825199865159
Train Epoch: 44/100 [iter:31/46], acc:0.9707661271095276, loss:0.08636142361548639
Train Epoch: 44/100 [iter:41/46], acc:0.9710365533828735, loss:0.08641810524390965
Train Epoch: 44/100 Traing_Loss: 0.09368735109103594 Traing_acc: 96.973869% 
Val_Loss: 0.1988707559959765 Val_accuracy: 95.640327%
1 epoch spends  12224.541863203049  seconds
Train Epoch: 45/100 [iter:1/46], acc:1.0, loss:0.013364389538764954
Train Epoch: 45/100 [iter:11/46], acc:0.9744318723678589, loss:0.05857241526246071
Train Epoch: 45/100 [iter:21/46], acc:0.9747024178504944, loss:0.06273410469293594
Train Epoch: 45/100 [iter:31/46], acc:0.9697580337524414, loss:0.06795511274568496
Train Epoch:

Train Epoch: 72/100 [iter:11/46], acc:0.980113685131073, loss:0.07847152921286496
Train Epoch: 72/100 [iter:21/46], acc:0.980654776096344, loss:0.0664881415487755
Train Epoch: 72/100 [iter:31/46], acc:0.9798386693000793, loss:0.06752883997415343
Train Epoch: 72/100 [iter:41/46], acc:0.9801828861236572, loss:0.06711158478950582
Train Epoch: 72/100 Traing_Loss: 0.0647754641723764 Traing_acc: 98.143051% 
Val_Loss: 0.21316826635875233 Val_accuracy: 94.550407%
1 epoch spends  13630.608324289322  seconds
Train Epoch: 73/100 [iter:1/46], acc:1.0, loss:0.01209431141614914
Train Epoch: 73/100 [iter:11/46], acc:0.9744318723678589, loss:0.06193653798916123
Train Epoch: 73/100 [iter:21/46], acc:0.9672619104385376, loss:0.07700642570853233
Train Epoch: 73/100 [iter:31/46], acc:0.9727822542190552, loss:0.06826015790143321
Train Epoch: 73/100 [iter:41/46], acc:0.9756097197532654, loss:0.06646733213125205
Train Epoch: 73/100 Traing_Loss: 0.06272153382124239 Traing_acc: 97.730400% 
Val_Loss: 0.22169856

Train Epoch: 100/100 [iter:41/46], acc:0.9733231067657471, loss:0.07282246526603292
Train Epoch: 100/100 Traing_Loss: 0.07283630499813547 Traing_acc: 97.180191% 
Val_Loss: 0.21025277061098602 Val_accuracy: 94.277931%
1 epoch spends  15058.55928492546  seconds
start predict.
***********************
0 th start tta predict.
5104.139165401459  seconds
********************************
Fold 2/5
Train Epoch: 1/100 [iter:1/46], acc:0.28125, loss:1.4512983560562134
Train Epoch: 1/100 [iter:11/46], acc:0.5625, loss:1.0513764511455188
Train Epoch: 1/100 [iter:21/46], acc:0.6726190447807312, loss:0.8662854049886975
Train Epoch: 1/100 [iter:31/46], acc:0.71875, loss:0.7595655245165671
Train Epoch: 1/100 [iter:41/46], acc:0.7454267740249634, loss:0.6919621381817794
Train Epoch: 1/100 Traing_Loss: 0.676107502245641 Traing_acc: 75.206047% 
Val_Loss: 0.31057338257358497 Val_accuracy: 89.041100%
1 epoch spends  15153.588185548782  seconds
Train Epoch: 2/100 [iter:1/46], acc:0.875, loss:0.420617759227752

Val_Loss: 0.17635789211482217 Val_accuracy: 93.972603%
1 epoch spends  15807.904164791107  seconds
Train Epoch: 15/100 [iter:1/46], acc:0.90625, loss:0.15602277219295502
Train Epoch: 15/100 [iter:11/46], acc:0.9403409361839294, loss:0.17335633323951202
Train Epoch: 15/100 [iter:21/46], acc:0.9330357313156128, loss:0.2272170271192278
Train Epoch: 15/100 [iter:31/46], acc:0.9264112710952759, loss:0.23477687590545224
Train Epoch: 15/100 [iter:41/46], acc:0.9291158318519592, loss:0.22836947277551745
Train Epoch: 15/100 Traing_Loss: 0.21973515834127152 Traing_acc: 93.337921% 
Val_Loss: 0.15950734909266642 Val_accuracy: 95.342468%
1 epoch spends  15857.94037270546  seconds
Train Epoch: 16/100 [iter:1/46], acc:0.9375, loss:0.3101799488067627
Train Epoch: 16/100 [iter:11/46], acc:0.9318181872367859, loss:0.19697135957804593
Train Epoch: 16/100 [iter:21/46], acc:0.9360119104385376, loss:0.17605605047373546
Train Epoch: 16/100 [iter:31/46], acc:0.9324596524238586, loss:0.18325530449228902
Train 

Train Epoch: 43/100 [iter:21/46], acc:0.9627976417541504, loss:0.10238144191957656
Train Epoch: 43/100 [iter:31/46], acc:0.9556451439857483, loss:0.11567060889736298
Train Epoch: 43/100 [iter:41/46], acc:0.9535060524940491, loss:0.11681027392425188
Train Epoch: 43/100 Traing_Loss: 0.11888748898610964 Traing_acc: 95.260986% 
Val_Loss: 0.15797043042640163 Val_accuracy: 94.794525%
1 epoch spends  17260.96054339409  seconds
Train Epoch: 44/100 [iter:1/46], acc:0.9375, loss:0.169901505112648
Train Epoch: 44/100 [iter:11/46], acc:0.9744318723678589, loss:0.10414357889782298
Train Epoch: 44/100 [iter:21/46], acc:0.9702380895614624, loss:0.10197563096880913
Train Epoch: 44/100 [iter:31/46], acc:0.9627015590667725, loss:0.11053702307324256
Train Epoch: 44/100 [iter:41/46], acc:0.9626523852348328, loss:0.10883452788722224
Train Epoch: 44/100 Traing_Loss: 0.10682362964847586 Traing_acc: 96.359894% 
Val_Loss: 0.1588727898793678 Val_accuracy: 94.246582%
1 epoch spends  17311.491428375244  seconds
T

In [37]:
def result_list(i, name):
    result = []
    for i_idx in i:
        result.append(i_idx)
    return pd.DataFrame(result, columns = [name])

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.0
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment='center', color='white' if cm[i, j] > thresh else 'black')

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predict label')
    
def visualization(train, valid, title, fold):
    plt.plot(range(num_epochs), train, 'b-', label = f'Training_{title}')
    plt.plot(range(num_epochs), valid, 'g-', label = f'validation{title}')
    plt.title(f'Training & Validation {title}')
    plt.xlabel('Number of epochs')
    plt.ylabel(title)
    plt.legend()
    plt.savefig(os.path.join(save_file, f"{title}_{fold}.png"))
    plt.show()


def all_confusion_matrix(val_target_value, val_predict_value, fold):

    labels_classes = ['healthy', 'multiple_diseases', 'rust', 'scab']

    val_target_value_ = []
    val_predict_value_ = []

    for i in val_target_value:
        for j in range(len(i)):
            val_target_value_.append(i[j])

    for i in val_predict_value:
        for j in range(len(i)):
            val_predict_value_.append(i[j])        
    
    val_micro = metrics.precision_score(val_target_value_, val_predict_value_, average='micro')
    val_macro = metrics.precision_score(val_target_value_, val_predict_value_, average='macro')
    print('val_micro', val_micro)
    print('val_macro', val_macro)
    
    cm = metrics.confusion_matrix(val_target_value_, val_predict_value_)
    plot_confusion_matrix(cm, labels_classes)
    plt.savefig(os.path.join(save_file, f"confusion_matrix_{fold}_TTA{TTA}.png"))
    plt.show()
    
    return val_micro, val_macro

In [ ]:
valid_metric = []

for i in train_results:
    fold = i['fold']
    train_acc = i['train_acc']
    train_loss = i['train_loss']
    val_acc = i['val_acc']
    val_loss = i['val_loss']
    
    print('Fold:', fold)
    train_acc_result = result_list(train_acc, 'train_acc')
    train_loss_result = result_list(train_loss, 'train_loss')
    val_acc_result = result_list(val_acc, 'val_acc')
    val_loss_result = result_list(val_loss, 'val_loss')
    train_valid_result = pd.concat([train_acc_result, train_loss_result, val_acc_result, val_loss_result], axis = 1)
    train_valid_result.to_csv(os.path.join(save_file, f'train_valid_result_{fold}_fold_TTA{TTA}.csv'))
    
    val_target_value = i['val_target_value']
    val_predict_value = i['val_predict_value']
    val_micro, val_macro = all_confusion_matrix(val_target_value, val_predict_value, fold, 1)
    visualization(train_acc, val_acc, 'Accuracy', fold, 1)
    visualization(train_loss, val_loss, 'Loss', fold, 1)
    valid_metric.append({'fold':fold, 'val_micro':val_micro, 'val_macro':val_macro})

valid_metric_df = pd.DatsFrame(valid_metric, columns=['fold', 'val_micro', 'val_macro'])
valid_metric_df.to_csv(os.path.join(save_file, f'valid_metric_df_TTA{TTA}.csv'), index=False)

In [ ]:
predict_df.to_csv(os.path.join(save_file, f'Result_tta{TTA}.csv'), index=False)

In [18]:
f_txt = open(os.path.join(save_file, "parameter.txt"), "w")

f_txt.write(f"model_name: {model_name}\n")
f_txt.write(f"batch_size: {batch_size}\n")
f_txt.write(f"N_FOLDS: {N_FOLDS}\n")
f_txt.write(f"TTA: {TTA}\n")

f_txt.close()